In [1]:
import pandas as pd
import numpy as np
import datetime
data_notnull_num = 60      ##非nan最少条数要求

In [2]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)


# dt = pd.read_csv('input/rpt_email_core_usr_match_d_cjg_0801.csv')
# direction = pd.read_csv('direction/rpt.rpt_email_core_usr_match_d.csv')

dt = pd.read_csv('input/rpt_email_core_usr_grw_d_cjg_0801.csv')
direction = pd.read_csv('direction/rpt.rpt_email_core_usr_grw_d.csv')

chain_index_set = pd.read_excel('监控指标设置处/监控指标表.xlsx',sheet_name='环比指标（相对百分比）')
chainAbsolute_index_set = pd.read_excel('监控指标设置处/监控指标表.xlsx',sheet_name='环比指标（绝对百分数）')
deviation_index_set = pd.read_excel('监控指标设置处/监控指标表.xlsx',sheet_name='偏离指标')
extreme_index_set = pd.read_excel('监控指标设置处/监控指标表.xlsx',sheet_name='极值指标')
continuous_index_set = pd.read_excel('监控指标设置处/监控指标表.xlsx',sheet_name='连续指标')

In [3]:
data_delta_index1 = list(chain_index_set['前几日'].unique())
month_delta_index1 = list(deviation_index_set['前几月'].unique())
extreme_delta_index1 = list(extreme_index_set['时间范围'].unique())

In [4]:
for i in dt.columns:
    if ((direction[direction['字段名']==i]['字段说明'].isnull())==False).iloc[0]:
        dt.rename(columns={i:direction[direction['字段名']==i]['字段说明'].iloc[0]},inplace=True)
    elif ((direction[direction['字段名']==i]['字段说明'].isnull())==True).iloc[0]:
        dt.drop([i],axis=1,inplace=True)
dt.rename(columns={'date':'day'},inplace=True)
dt.day = pd.to_datetime(dt.day)

In [5]:
conclusion_data_columns = ['day']
for data_delta_i in data_delta_index1:
    s = 'day_minus_'+str(data_delta_i)
    conclusion_data_columns.append(s)
conclusion_data_columns.append('备注')
conclusion_data = pd.DataFrame(columns=conclusion_data_columns)

In [6]:
def deal_data(data_1,data_2,dt_delta,absolute_percent):

    data_2_columns = data_2.columns[0]
    data = pd.concat([data_1,data_2],axis=1)

    # # 环比指标/同比指标（相对百分比）
    res_1 = pd.DataFrame(index=[data_2_columns])

    T_raw = data.day.max()
    T = pd.to_datetime(T_raw)-datetime.timedelta(days=(dt_delta)) 
    T_data = (data[data['day']==T].iloc[0,1])
    res_1['day'] = T_data

    for data_delta_i in data_delta_index1:
        s= pd.to_datetime(T)-datetime.timedelta(days=(np.int(data_delta_i)))
        locals() ['T_minus'+str(data_delta_i)+'_data'] = (data[data['day']==s].iloc[0,1])
        res_1['day_minus_'+str(data_delta_i)] = (data[data['day']==s].iloc[0,1])
        T_minus_data = locals() ['T_minus'+str(data_delta_i)+'_data']
        for index_i in range(chain_index_set.shape[0]):
            chain_index_FieldName = chain_index_set.loc[index_i,'字段名称']
            chain_index_IndexName = chain_index_set.loc[index_i,'指标名称']
            chain_index_UpperThreshold = chain_index_set.loc[index_i,'阈值上限']
            chain_index_LowerThreshold = chain_index_set.loc[index_i,'阈值下限']
            chain_index_DayAgo = chain_index_set.loc[index_i,'前几日']
            if (T_data>=T_minus_data*chain_index_LowerThreshold)&(T_data<T_minus_data*chain_index_UpperThreshold)\
            &(chain_index_IndexName !=-999)&(chain_index_DayAgo == data_delta_i):
                res_1[chain_index_FieldName] = chain_index_IndexName+'{:.2%}'.format(T_data/T_minus_data-1)
            elif (T_data>=T_minus_data*chain_index_LowerThreshold)&(T_data<T_minus_data*chain_index_UpperThreshold)\
            &(chain_index_IndexName ==-999)&(chain_index_DayAgo == data_delta_i):
                res_1[chain_index_FieldName] = -999

    # # 环比指标/同比指标（绝对百分数）
    if absolute_percent==True:
        for data_delta_i in data_delta_index1:
            s= pd.to_datetime(T)-datetime.timedelta(days=(np.int(data_delta_i)))
            locals() ['T_minus'+str(data_delta_i)+'_data'] = (data[data['day']==s].iloc[0,1])
            res_1['day_minus_'+str(data_delta_i)] = (data[data['day']==s].iloc[0,1])
            T_minus_data = locals() ['T_minus'+str(data_delta_i)+'_data']
            for index_i in range(chainAbsolute_index_set.shape[0]):
                chain_index_FieldName = chainAbsolute_index_set.loc[index_i,'字段名称']
                chain_index_IndexName = chainAbsolute_index_set.loc[index_i,'指标名称']
                chain_index_UpperThreshold = chainAbsolute_index_set.loc[index_i,'阈值上限']
                chain_index_LowerThreshold = chainAbsolute_index_set.loc[index_i,'阈值下限']
                chain_index_DayAgo = chainAbsolute_index_set.loc[index_i,'前几日']
                if (T_data>=T_minus_data+chain_index_LowerThreshold)&(T_data<T_minus_data+chain_index_UpperThreshold)\
                &(chain_index_IndexName !=-999)&(chain_index_DayAgo == data_delta_i):
                    res_1[chain_index_FieldName] = chain_index_IndexName+'{:.2%}'.format(T_data-T_minus_data)
                elif (T_data>=T_minus_data+chain_index_LowerThreshold)&(T_data<T_minus_data+chain_index_UpperThreshold)\
                &(chain_index_IndexName ==-999)&(chain_index_DayAgo == data_delta_i):
                    res_1[chain_index_FieldName] = -999


    # # 偏离指标

    def Year_Month(year_input,month_input,month_delta):
        year_month_count = year_input * 12 + month_input
        year_month_count_end = year_month_count - 1
        year_month_count_start = year_month_count - month_delta

        year_end = year_month_count_end // 12
        month_end = year_month_count_end % 12
        year_start = year_month_count_start // 12
        month_start = year_month_count_start % 12

        return year_end, month_end, year_start, month_start

    res_2 = pd.DataFrame(index=[data_2_columns])

    data['month'] = pd.to_datetime(data['day']).apply(lambda x:x.month)
    data['year'] = pd.to_datetime(data['day']).apply(lambda x:x.year)
    for month_delta_i in month_delta_index1:
        Year_input = data['year'].max()
        Month_input = data['month'].max()
        year_end, month_end, year_start, month_start = Year_Month(Year_input,Month_input,month_delta_i)
        locals() ['miu_'+str(month_delta_i)] = data[((data['month']<=month_end)&(data['year']==year_end))&((data['month']>=month_start)&(data['year']==year_start))][data_2_columns].mean()
        locals() ['std_'+str(month_delta_i)] = data[((data['month']<=month_end)&(data['year']==year_end))&((data['month']>=month_start)&(data['year']==year_start))][data_2_columns].std()
        miu = locals() ['miu_'+str(month_delta_i)]
        std = locals() ['std_'+str(month_delta_i)]
        for index_i in range(deviation_index_set.shape[0]):
            deviation_index_FieldName = deviation_index_set.loc[index_i,'字段名称']
            deviation_index_IndexName = deviation_index_set.loc[index_i,'指标名称']
            deviation_index_UpperThreshold = deviation_index_set.loc[index_i,'阈值上限']
            deviation_index_LowerThreshold = deviation_index_set.loc[index_i,'阈值下限']
            deviation_index_MonthAgo = deviation_index_set.loc[index_i,'前几月']
            if (T_data>=miu+deviation_index_LowerThreshold*std)&(T_data<miu+deviation_index_UpperThreshold*std)\
            &(deviation_index_MonthAgo==month_delta_i):
                res_2[deviation_index_FieldName] = deviation_index_IndexName


    # # 极值指标
    res_3 = pd.DataFrame(index=[data_2_columns])
    for extreme_delta_i in extreme_delta_index1:
        s = pd.to_datetime(T)-datetime.timedelta(days=(np.int(extreme_delta_i)))
        s_max = (data[data['day']>=s].iloc[:,1].max())
        s_min = (data[data['day']>=s].iloc[:,1].min())
#         print(T_data,s_max,s_min,extreme_delta_i)
        for index_i in range(extreme_index_set.shape[0]):
            extreme_index_FieldName = extreme_index_set.loc[index_i,'字段名称']
            extreme_index_IndexName = extreme_index_set.loc[index_i,'指标名称']
            extreme_index_TimeRange = extreme_index_set.loc[index_i,'时间范围']
            extreme_index_Flag = extreme_index_set.loc[index_i,'flag(大1小-1)']        
            if (T_data>=s_max)&(extreme_delta_i==extreme_index_TimeRange)&(extreme_index_Flag==1):
                res_3[extreme_index_FieldName] = extreme_index_IndexName
            elif (T_data<s_min)&(extreme_delta_i==extreme_index_TimeRange)&(extreme_index_Flag==-1):
                res_3[extreme_index_FieldName] = extreme_index_IndexName
            if (T_data<s_max)&(T_data>=s_min)&(extreme_delta_i==extreme_index_TimeRange)&(extreme_index_Flag==0):
                res_3[extreme_index_FieldName] = extreme_index_IndexName


    # # 连续指标
    res_4 = pd.DataFrame(index=[data_2_columns])

    cnt_increase = 0
    cnt_decrease = 0
    # T = data.day.max()
    for i in range(1,len(data)-10):
        delta = data[data['day']==pd.to_datetime(T)-datetime.timedelta(days=(i))].iloc[0,1] - data[data['day']==pd.to_datetime(T)-datetime.timedelta(days=(i+1))].iloc[0,1]
        if delta >=0:
            cnt_increase += 1
        elif delta < 0:
            break
    for j in range(1,len(data)-10):
        delta = data[data['day']==pd.to_datetime(T)-datetime.timedelta(days=(j))].iloc[0,1] - data[data['day']==pd.to_datetime(T)-datetime.timedelta(days=(j+1))].iloc[0,1]
        if delta < 0:
            cnt_decrease += 1
        elif delta >= 0:
            break

    for index_i in range(continuous_index_set.shape[0]):
        continuous_index_FieldName = continuous_index_set.loc[index_i,'字段名称']
        continuous_index_IndexName = continuous_index_set.loc[index_i,'指标名称']
        continuous_index_Threshold = continuous_index_set.loc[index_i,'阈值（希望连续大于多少天则预警）']
        continuous_index_Flag = continuous_index_set.loc[index_i,'flag(涨1降-1)']   
        if (cnt_increase>=continuous_index_Threshold)&(continuous_index_Flag==1):
            res_4[continuous_index_FieldName] = continuous_index_IndexName+str(cnt_increase)+'天'
        elif (cnt_increase<continuous_index_Threshold)&(continuous_index_Flag==1):
            res_4[continuous_index_FieldName] = -999
        elif (cnt_decrease>=continuous_index_Threshold)&(continuous_index_Flag==-1):
            res_4[continuous_index_FieldName] = continuous_index_IndexName+str(cnt_decrease)+'天'
        elif (cnt_decrease<continuous_index_Threshold)&(continuous_index_Flag==-1):
            res_4[continuous_index_FieldName] = -999

    # # 汇总指标
    res = pd.concat([res_1,res_2],axis=1)
    res = pd.concat([res,res_3],axis=1)
    res = pd.concat([res,res_4],axis=1)

    data_delta_columns_minus = []
    for data_delta_i in data_delta_index1:
        data_delta_columns_minus.append('day_minus_'+str(data_delta_i))
    data_delta_columns_minus.append('day')

    res_conclusion_index_before=res.columns
    res_conclusion_index_before = res_conclusion_index_before.drop(data_delta_columns_minus)
    res_conclusion_index_after = ''
    for i in res_conclusion_index_before:
        if res[i][0]!=-999:
            res_conclusion_index_after = res_conclusion_index_after+str(res[i][0]+', ')

    res_conclusion = res[data_delta_columns_minus].copy()
    res_conclusion['备注'] = res_conclusion_index_after

    return res_conclusion

In [7]:
data_delta_columns_minus = ['day']
for data_delta_i in data_delta_index1:
    data_delta_columns_minus.append('day_minus_'+str(data_delta_i))
data_delta_columns_minus.append('备注')

data_1=dt[['day']]
for i in dt.columns:
    if i !='day':
        if ((direction[direction['字段说明']==i]['指标类型'])==3).iloc[0]:
            if '次日' in i:
                dt_delta_value = 1
            elif '7日' in i:
                dt_delta_value = 7
            elif '30日' in i:
                dt_delta_value = 30
            else:
                dt_delta_value = 0
            if dt[i].notnull().sum()>=data_notnull_num:
                dt[i].fillna('-99900%',inplace=True)
                dt[i] = dt[i].apply(lambda x:np.float(x.strip("%"))/100)
                absolute_percent = True
                data_2=dt[[i]]
                s=deal_data(data_1,data_2,dt_delta_value,absolute_percent)
                conclusion_data=pd.concat([conclusion_data,s],axis=0)
            elif dt[i].notnull().sum()<data_notnull_num:
                s = pd.DataFrame(columns=data_delta_columns_minus,index=[i])
                for i in conclusion_data.columns:
                    s[i]='有效数据量太少，不足'+str(data_notnull_num)+'条'
                conclusion_data=pd.concat([conclusion_data,s],axis=0)
        elif (((direction[direction['字段说明']==i]['指标类型'])==1).iloc[0])|(((direction[direction['字段说明']==i]['指标类型'])==2).iloc[0]):
            if '次日' in i:
                dt_delta_value = 1
            elif '7日' in i:
                dt_delta_value = 7
            elif '30日' in i:
                dt_delta_value = 30
            else:
                dt_delta_value = 0
            if dt[i].notnull().sum()>=data_notnull_num:
                dt[i].fillna(-999,inplace=True)
                absolute_percent = False
                data_2=dt[[i]]
                s=deal_data(data_1,data_2,dt_delta_value,absolute_percent)
                conclusion_data=pd.concat([conclusion_data,s],axis=0)
            elif dt[i].notnull().sum()<data_notnull_num:
                s = pd.DataFrame(columns=data_delta_columns_minus,index=[i])
                for i in conclusion_data.columns:
                    s[i]='有效数据量太少，不足'+str(data_notnull_num)+'条'                
                conclusion_data=pd.concat([conclusion_data,s],axis=0)                
            

In [8]:
conclusion_data.to_csv('output/conclusion_data.csv')

In [9]:
conclusion_data

,day,day_minus_1,day_minus_7,day_minus_30,备注
DAU,6403477,6332338,6027620,5505805,"环比30日前上涨（相对百分比）16.30%, 上分位数>=97.72%（近上个完整自然月）,..."
新增用户数,299036,253770,237293,164691,"环比昨日上涨（相对百分比）17.84%, 环比7日前上涨（相对百分比）26.02%, 环比3..."
Android新增用户数,250429,210821,202330,128167,"环比昨日上涨（相对百分比）18.79%, 环比7日前上涨（相对百分比）23.77%, 环比3..."
iOS新增用户数,48607,42949,34963,36524,"环比昨日上涨（相对百分比）13.17%, 环比7日前上涨（相对百分比）39.02%, 环比3..."
新增用户中男性占比,0.5976,0.6036,0.6311,0.5969,"下分位数<=17.37%（近上个完整自然月）,"
新增用户中女性占比,0.4023,0.3964,0.3688,0.403,"上分位数>=82.63%（近上个完整自然月）,"
活跃用户中男性占比,0.6331,0.6335,0.6366,0.6407,"下分位数<=17.37%（近上个完整自然月）,"
活跃用户中女性占比,0.3668,0.3664,0.3633,0.3592,"上分位数>=82.63%（近上个完整自然月）, 连续上涨天数：3天,"
WAU,12620121,12468134,11795993,10575521,"环比30日前上涨（相对百分比）19.33%, 上分位数>=97.72%（近上个完整自然月）,..."
MAU,21451458,21243600,20241242,17201016,"环比30日前上涨（相对百分比）24.71%, 上分位数>=82.63%（近上个完整自然月）,..."


In [26]:
def gen_user_nunique_features(df, value, prefix):
    group_df = df.groupby(['user']).agg(
        s=locals()['user_'+str(1)+'_'+str(1)+'_nuniq'],
        s=pd.NamedAgg(column=1, aggfunc='nunique'),
    ).reset_index()
    return group_df

SyntaxError: keyword argument repeated (<ipython-input-26-4bdf6a2d46cc>, line 4)

In [23]:
locals()['user_'+str(1)+'_'+str(1)+'_nuniq']=pd.NamedAgg(column=1, aggfunc='nunique'),